In [ ]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
%matplotlib inline
import  matplotlib.pyplot as plt
import torch
from torch import  nn,optim

import random
import torch
import os

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(seed=521)

In [ ]:
train= pd.read_excel('./dataset/test.xlsx')
test= pd.read_excel('./dataset/test.xlsx')


time_columns=['time']
X_columns = [col for col in train.columns if col not in ['time','TNeff']]
Y_columns =['TNeff']

x_train=train[X_columns]
y_train=train[Y_columns]
x_test=test[X_columns]
y_test=test[Y_columns]

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("*********************")
print(torch.cuda.get_device_name(0))
print(device)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler1=MinMaxScaler(feature_range=(0.000000001,1))
scaler2=MinMaxScaler(feature_range=(0.000000001,1))
scaler3=MinMaxScaler(feature_range=(0.000000001,1))
scaler4=MinMaxScaler(feature_range=(0.000000001,1))
train_sca=pd.DataFrame()
test_sca=pd.DataFrame()
train_sca[X_columns]=scaler1.fit_transform(train[X_columns])
train_sca[Y_columns]=scaler2.fit_transform(train[Y_columns])
test_sca[X_columns]=scaler3.fit_transform(test[X_columns])
test_sca[Y_columns]=scaler4.fit_transform(test[Y_columns])




In [ ]:
input_size=len(X_columns)
output_size=len(Y_columns)

hidden_size=16
num_layers=8

seq=16
batch_size=4

epochs=100





In [ ]:
def split_data(train_sca,test_sca,seq,X_columns,Y_columns): #47预测1


    data_raw=pd.concat([train_sca,test_sca])

    data=[]
    for index in range(len(data_raw)-seq):
        data.append(data_raw[index:index+seq])
    data=np.array(data)
    # print(len(data))  #14640
    test_set_size=len(test_sca)  #96
    train_set_size=data.shape[0]-test_set_size

    # 第一个-1是47预测1
    #第二个1是不计time，-1是只预测AQ
    x_train=data[:train_set_size,:-1,:len(X_columns)].astype('float32')
    y_train=data[:train_set_size,-1,-len(Y_columns)].reshape(-1,len(Y_columns)).astype('float32')

    x_test=data[train_set_size:,:-1,:len(X_columns)].astype('float32')
    y_test=data[train_set_size:,-1,-len(Y_columns)].reshape(-1,len(Y_columns)).astype('float32')

    def nptotorch(inputdata):
        return torch.from_numpy(inputdata).to(device)
    x_train_ts= nptotorch(x_train)
    y_train_ts= nptotorch(y_train)
    x_test_ts= nptotorch(x_test)
    y_test_ts= nptotorch(y_test)

    return [x_train_ts,y_train_ts,x_test_ts,y_test_ts]


x_train_ts,y_train_ts,x_test_ts,y_test_ts=split_data(
    train_sca=train_sca,
    # train_sca=train.iloc[:,1:],
    test_sca=test_sca,
    # test_sca=test.iloc[:,1:],
    seq=seq,
    X_columns=X_columns,
    Y_columns=Y_columns)
print(x_train_ts.shape)
print(y_train_ts.shape)
print(x_test_ts.shape)
print(y_test_ts.shape)

In [ ]:
import  torch.nn as nn
class GRU(nn.Module):
    def __init__(self,input_size,hidden_size,num_layers,output_size):
        super(GRU,self).__init__()

        self.input_size=input_size
        self.hidden_size=hidden_size
        self.num_layers=num_layers
        self.output_size=output_size


        self.GRU=nn.GRU(self.input_size,self.hidden_size,self.num_layers,batch_first=True,dropout=0.5)
        self.fc=nn.Linear(self.hidden_size,self.output_size)


    def forward(self,input):
        # batch_size, seq_len = train.shape[0],train.shape[1]

        # h0=torch.zeros(self.num_layers,batch_size,self.hidden_size).to(device)
        # c0=torch.zeros(self.num_layers,batch_size,self.hidden_size).to(device)
        # out,(hn,cn)=self.lstm(train,(h0.detach(),c0.detach()))

        # h0=torch.zeros(self.num_layers,batch_size,self.hidden_size).to(device)
        # c0=torch.zeros(self.num_layers,batch_size,self.hidden_size).to(device)
        out,_=self.GRU(input)
        out = self.fc(out)
        output = out[:, -1,:]


        return output



In [ ]:
model =GRU(input_size=input_size,hidden_size=hidden_size,output_size=output_size,num_layers=num_layers).to(device)


# def MAPE(y_true, y_pred):
#     with np.errstate(divide='ignore', invalid='ignore'):
#         if np.isnan(null_val):
#             mask = ~np.isnan(y_t)
#         else:
#             mask = np.not_equal(y_t, null_val)
#         mask = mask.astype('float32')
#         mask /= np.mean(mask)
#         mape = np.abs(np.divide(np.subtract(y_p, y_t).astype('float32'),
#                       y_t))
#         mape = np.nan_to_num(mask * mape)
#         return np.mean(mape)

myloss=nn.MSELoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)


In [ ]:
import time
start_time=time.time()
hist=np.zeros(epochs)

lstm=[]

from torch.utils import data
torch_dataset = data.TensorDataset(x_train_ts, y_train_ts)
train_iter=data.DataLoader(
    dataset=torch_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0
)



In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
# lossmin=1
# for epoch in range(epochs):
#     for inx,(X, Y) in enumerate(train_iter):
#
#         Y_pred=model(X)
#         loss =myloss(Y,Y_pred)
#
#
#         #梯度清0
#         optimizer.zero_grad()
#         #计算梯度
#         loss.backward()
#         #修改权值
#         optimizer.step()
#
#
#         torch.cuda.empty_cache()
#
#
#
#
#     test_index=test.index
#     y_test_pred_ts = model(x_test_ts)
#     y_test_pred = y_test_pred_ts.detach().cpu().numpy().reshape(-1,1)
#     y_test_pred=scaler4.inverse_transform(y_test_pred)
#     loss=mean_absolute_percentage_error(test[Y_columns],y_test_pred)
#     print(f'*****************epoch{epoch} MAPE {loss}')
#     if loss<1 and loss<lossmin:
#         lossmin=loss
#         torch.save(model.state_dict(),"./mpkl/m14_gru_.pkl")
#
#
#
# training_time=time.time()-start_time
# print(f'training_time{training_time}')

model.load_state_dict(torch.load("./mpkl/m14_gru_.pkl"))



train_index=train.index
y_train_pred_ts = model(x_train_ts)
y_train_pred = y_train_pred_ts.detach().cpu().numpy().reshape(-1,1)
y_train_pred=scaler2.inverse_transform(y_train_pred)
y_train_pred=pd.DataFrame(columns=['AQ'],data=y_train_pred)

test_index=test.index
y_test_pred_ts = model(x_test_ts)
y_test_pred = y_test_pred_ts.detach().cpu().numpy().reshape(-1,1)
y_test_pred=scaler4.inverse_transform(y_test_pred)
y_test_pred=pd.DataFrame(columns=['AQ'],data=y_test_pred)
print(f'***************************************************')
loss=mean_absolute_percentage_error(test[Y_columns],y_test_pred)
print('MAPE:', loss)

training_time=time.time()-start_time
print(f'training_time{training_time}')

In [ ]:
import seaborn as sns
sns.set(context='paper', style='ticks')

fig,axes=plt.subplots(2,1,figsize=(20,10))
ax1=axes[0]
ax2=axes[1]
fig.subplots_adjust(left=0.1, right=0.8, bottom=0.1, top=0.9, wspace=0.2, hspace=0.3)


# x=train_index[::32]
# y1=y_train['AQ'][::32]
# y2=y_train_pred['AQ'][::32]
lns1=ax1.plot(y_train[:],label='y_train',marker='v',markevery=1, color='lightcoral')
lns2=ax1.plot(y_train_pred[:],label='y_train_pred',marker='D',markevery=1, color='dodgerblue')
ax1.tick_params(labelsize=12)
# ax1.set_xlim(0,14208,2000)
# ax1.set_ylim(0,35000, 5000)
# ax1.set_xlabel('Timeindex(8h)',size=20)
# ax1.set_ylabel('AQ(m³/h)',size=20)
ax1.set_title('m1_ridge',size=20,fontweight='bold',loc='left')
lns = lns1+lns2
labs = [l.get_label() for l in lns]
ax1.legend(lns, labs, loc=1,fontsize=12,ncol=2,frameon=False)



lns1=ax2.plot(y_test[:],label='y_test',marker='v',markevery=1, color='lightcoral')
lns2=ax2.plot(y_test_pred[:],label='y_test_pred',marker='D',markevery=1, color='dodgerblue')
ax2.tick_params(labelsize=12)
# ax1.set_xlim(0,14208,2000)
# ax1.set_ylim(0,35000, 5000)
# ax1.set_xlabel('Timeindex(8h)',size=20)
# ax1.set_ylabel('AQ(m³/h)',size=20)
ax2.set_title('m1_ridge',size=20,fontweight='bold',loc='left')
lns = lns1+lns2
labs = [l.get_label() for l in lns]
ax2.legend(lns, labs, loc=1,fontsize=12,ncol=2,frameon=False)

In [ ]:
frame=pd.DataFrame()
frame['ytrain']=y_train_pred
frame['ytest']=y_test_pred
frame.to_excel('./modelresult/m14_gru.xlsx')

